In [2]:
import warnings
warnings.filterwarnings('ignore')
import glob
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import OneHotEncoder
import random
import os
from sklearn.metrics import log_loss

In [3]:
d = "C:\kaggle_data\credit_card"
lst = os.listdir(d)
print(lst)
train = pd.read_csv(d + '\\' +lst[3])
test = pd.read_csv(d + '\\' +lst[2])
ss = pd.read_csv(d + '\\' +lst[1])
train = train.drop(['index'], axis=1)
train.fillna('NAN', inplace=True) 
test = test.drop(['index'], axis=1)
test.fillna('NAN', inplace=True)

['.ipynb_checkpoints', 'sample_submission.csv', 'test.csv', 'train.csv', 'Untitled.ipynb']


# 전처리

### family_type
- 'Married', 'Civil marriage'
- 'Separated',  'Widow'
- 'Single / not married'

In [4]:
# Married, Civil marriage
train['income_per_size'] = np.log(train['income_total']/train['family_size'])
test['income_per_size'] = np.log(test['income_total']/test['family_size'])
train.loc[(train['family_type']=='Married')|(train['family_type']=='Civil marriage'),'income_per_size']\
= train['income_per_size'] * 2

test.loc[(test['family_type']=='Married')|(test['family_type']=='Civil marriage'),'income_per_size']\
= test['income_per_size'] * 2

In [5]:
def simple_marry(x):
    if x == 'Married' or x =='Civil marriage':
        return '0'
    elif x == 'Separated' or x == 'Widow':
        return '1'
    else:
        return '2'

In [6]:
# 안 덮어쓰기
for df in [train,test]:
    df['family_bins'] = df['family_type'].apply(simple_marry)

In [7]:
# income_total을 로그변환
train['income_total'] = np.log(train['income_total'])
test['income_total'] = np.log(test['income_total'])

In [8]:
# car와 reality를 합친 새로운 칼럼 careality
train['car'] =train['car'].apply(lambda x: int(x=='Y'))
train['reality'] =train['reality'].apply(lambda x: int(x=='Y'))
test['car'] =test['car'].apply(lambda x: int(x=='Y'))
test['reality'] =test['reality'].apply(lambda x: int(x=='Y'))

train['careality'] = train['car'] + train['reality']
train = train.drop(['car', 'reality'],1)

test['careality'] = test['car'] + test['reality']
test = test.drop(['car', 'reality'],1)

In [9]:
object_col = []
for col in train.columns:
    if train[col].dtype == 'object':
        object_col.append(col)

In [10]:
enc = OneHotEncoder()
enc.fit(train.loc[:,object_col])


train_onehot_df = pd.DataFrame(enc.transform(train.loc[:,object_col]).toarray(), 
             columns=enc.get_feature_names(object_col))
train.drop(object_col, axis=1, inplace=True)
train = pd.concat([train, train_onehot_df], axis=1)

In [11]:
test_onehot_df = pd.DataFrame(enc.transform(test.loc[:,object_col]).toarray(), 
             columns=enc.get_feature_names(object_col))
test.drop(object_col, axis=1, inplace=True)
test = pd.concat([test, test_onehot_df], axis=1)

### 
- income_total
- begin_month
- DAYS_BIRTH

In [12]:
c = 'income_total'
k = 2.2
mean = train[c].mean()
std = train[c].std()
idxs = train.loc[(train[c]>= mean + k*std)|\
                (train[c]<= mean - k*std)].index
train = train.drop(idxs).reset_index(drop=True)

In [13]:
c = 'DAYS_BIRTH'
k = 2.1
mean = train[c].mean()
std = train[c].std()
idxs = train.loc[(train[c]>= mean + k*std)|\
                (train[c]<= mean - k*std)].index
train = train.drop(idxs).reset_index(drop=True)

In [14]:
# out: 아웃라이어
out_train = train.loc[train.DAYS_EMPLOYED>0]
in_train = train.loc[train.DAYS_EMPLOYED<=0]


out_test = test.loc[test.DAYS_EMPLOYED>0]
in_test = test.loc[test.DAYS_EMPLOYED<=0]

<!-- ###
- DAYS_EMPLOYED -->

In [15]:
# c = 'DAYS_EMPLOYED'
# k = 2.8
# mean = in_train[c].mean()
# std = in_train[c].std()
# idxs = in_train.loc[(in_train[c]>= mean + k*std)|\
#                 (in_train[c]<= mean - k*std)].index
# in_train = in_train.drop(idxs).reset_index(drop=True)

# 모델링, 학습

In [16]:
out_train = out_train.reset_index()
in_train = in_train.reset_index()

out_test = out_test.reset_index()
in_test = in_test.reset_index()

In [17]:
idx_out_train = out_train.pop('index')
idx_out_test = out_test.pop('index')

idx_in_train = in_train.pop('index')
idx_in_test = in_test.pop('index')

In [18]:
len(train.columns)

58

In [19]:
out_train.drop('DAYS_EMPLOYED',inplace=True,axis=1)
out_test.drop('DAYS_EMPLOYED',inplace=True,axis=1)

In [20]:
out_train.head()

,child_num,income_total,DAYS_BIRTH,FLAG_MOBIL,work_phone,phone,email,family_size,begin_month,credit,...,occyp_type_NAN,occyp_type_Private service staff,occyp_type_Realty agents,occyp_type_Sales staff,occyp_type_Secretaries,occyp_type_Security staff,occyp_type_Waiters/barmen staff,family_bins_0,family_bins_1,family_bins_2
0,0,11.967181,-22922,1,0,1,0,1.0,-41.0,2.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0,12.283034,-23113,1,0,0,0,2.0,-37.0,2.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0,11.967181,-21253,1,0,1,0,2.0,-7.0,2.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0,12.218495,-22361,1,0,1,0,1.0,-5.0,2.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0,11.813030,-21100,1,0,1,0,1.0,-24.0,2.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [21]:
def tr(train):
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    folds=[]
    losses=[]
    for train_idx, valid_idx in skf.split(train, train['credit']):
        folds.append((train_idx, valid_idx))
    random.seed(42)
    lgb_models={}
    for fold in range(5):
        print(f'===================================={fold+1}============================================')
        train_idx, valid_idx = folds[fold]
        X_train, X_valid, y_train, y_valid = train.drop(['credit'],axis=1).iloc[train_idx].values, train.drop(['credit'],axis=1).iloc[valid_idx].values,\
                                             train['credit'][train_idx].values, train['credit'][valid_idx].values 
        lgb = LGBMClassifier(n_estimators=1000)
        lgb.fit(X_train, y_train, 
                eval_set=[(X_train, y_train), (X_valid, y_valid)], 
                early_stopping_rounds=30,
               verbose=100)
        lgb_models[fold]=lgb
        losses.append(log_loss(y_valid, lgb.predict_proba(X_valid)))
        print(f'================================================================================\n\n')
    ls = sum(losses)/5
    print(ls)
    return ls, lgb_models

In [22]:
out_loss, out_models = tr(out_train)

====================================1============================================
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[46]	training's multi_logloss: 0.536846	valid_1's multi_logloss: 0.730823


====================================2============================================
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[67]	training's multi_logloss: 0.465484	valid_1's multi_logloss: 0.733352


====================================3============================================
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[41]	training's multi_logloss: 0.548809	valid_1's multi_logloss: 0.747471


====================================4============================================
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[65]	training's multi_logloss: 0.477421	valid_1's multi_logl

In [143]:
# # 하기 전
# 0.7473309710312624    # 365243
# 0.7325143920950408

############ income_total 줄어들었음
# 0.7394330741897761 0.008감소
# 0.7296875809525057 0.03감소

## begin_month
# 0.7473008353598098 감소
# 0.7336974090198503 증가

############ DAYS_BIRTH
# 0.7429028769292156 0.005 감소
# 0.7325143765246345 0.0000002 감소

## employed
# 0.7473309710312624 
# 0.7327501285387077 증가

########### income_total, days_birth
# 0.7412269713675264 0.006 감소
# 0.7296875809525057 0.003 감소









## DAYS_BIRTH
# 0.7429028769292156 0.005 감소
# 0.7325143765246345 0.0000002 감소

## income_total 줄어들었음
# 0.7394330741897761 감소
# 0.7296875809525057 감소



In [23]:
in_loss, in_models = tr(in_train)

====================================1============================================
Training until validation scores don't improve for 30 rounds
[100]	training's multi_logloss: 0.619915	valid_1's multi_logloss: 0.744151
[200]	training's multi_logloss: 0.520925	valid_1's multi_logloss: 0.732577
Early stopping, best iteration is:
[196]	training's multi_logloss: 0.524859	valid_1's multi_logloss: 0.732257


====================================2============================================
Training until validation scores don't improve for 30 rounds
[100]	training's multi_logloss: 0.617449	valid_1's multi_logloss: 0.744036
[200]	training's multi_logloss: 0.519304	valid_1's multi_logloss: 0.734981
Early stopping, best iteration is:
[254]	training's multi_logloss: 0.479195	valid_1's multi_logloss: 0.73341


====================================3============================================
Training until validation scores don't improve for 30 rounds
[100]	training's multi_logloss: 0.616044	valid_1

In [ ]:
print(in_loss)

In [24]:
in_zeros = np.zeros([len(in_test),3])
for fold in range(5):
    in_zeros += in_models[fold].predict_proba(in_test)/5
in_output = pd.DataFrame(in_zeros)
in_output = in_output.reindex(index=pd.Index(idx_in_test))

In [ ]:
out_zeros = np.zeros([len(out_test),3])
for fold in range(5):
    out_zeros += out_models[fold].predict_proba(out_test)/5
in_output = pd.DataFrame(in_zeros)
in_output = in_output.reindex(index=pd.Index(idx_in_test))

In [186]:
in2 = np.concatenate((in_zeros,np.array(idx_in_test).reshape(-1,1)),axis=1)
out2 = np.concatenate((out_zeros,np.array(idx_out_test).reshape(-1,1)),axis=1)

In [187]:
output = pd.DataFrame(np.concatenate((in2,out2),axis=0),columns=[0,1,2,'index'])

In [188]:
output['index'] = output['index'].astype('int')

In [196]:
ss = pd.read_csv(d + '\\' +lst[1])

In [199]:
output

,0,1,2,index
0,0.254328,0.176459,0.569213,1
1,0.037702,0.077119,0.885179,2
2,0.150319,0.105418,0.744263,3
3,0.074441,0.168475,0.757084,4
4,0.057947,0.112467,0.829586,5
...,...,...,...,...
9995,0.045256,0.062820,0.891924,9968
9996,0.113289,0.091170,0.795541,9981
9997,0.162660,0.185026,0.652315,9988
9998,0.231756,0.133885,0.634359,9992


In [200]:
for i in range(len(output)):
    row = output.loc[output['index']==i]
    ss.iloc[i,1:] = row.iloc[:,:3]

In [204]:
ss.to_csv('two_models.csv', index=False)

In [205]:
output.head()

,0,1,2,index
0,0.254328,0.176459,0.569213,1
1,0.037702,0.077119,0.885179,2
2,0.150319,0.105418,0.744263,3
3,0.074441,0.168475,0.757084,4
4,0.057947,0.112467,0.829586,5


In [206]:
output.tail()

,0,1,2,index
9995,0.045256,0.062820,0.891924,9968
9996,0.113289,0.091170,0.795541,9981
9997,0.162660,0.185026,0.652315,9988
9998,0.231756,0.133885,0.634359,9992
9999,0.083838,0.910228,0.005934,9994
